# Bibliotecas utilizadas

In [11]:
import pandas as pd
import requests
import bs4
import time

from IPython.display import IFrame

# Coleta de dados: Web Scraping

## Minha identificação ao usar o navegador para acessar sites com o Google Chrome

In [12]:
# Identificação do "Usuário agente"
agente = 'xxxxxxxxxxxxxxxxxxxxxxxxx'

# Criando chave de identificação
headers = { 'User-Agente': agente }

## Processo de scraping

### **Coletando HTML de site**
<font size = '3'>
Essa atividade será executada pela função 'htmlSite', que fará a requisição do código HTML do site e ajustá-lo.<br>

Obs:<br>

Para que a função funcione, é necessário o Usuário agente (headers) do seu navegador (Chrome), para que <br>
possa ser requerido o conteúdo do site, que virá em formato de byte. A partir disso transformamos o <br>
formato byte usando a codificação correta, tornando-o texto e, em seguida, estruturando-o como objeto <br>
BeautifulSoup, para futuras consultas de tags.    
</font>

In [13]:
# Função para extrair o código HTML do site de interesse
def htmlSite( url: str, decode: str = 'latin1' ) -> str:

    try:
        # Fazendo requisição de acesso ao html do link
        requisicao = requests.get( url, headers = headers, timeout = 4 )

        # Conteúdo do site ilegível e em formato byte
        resposta_bytes = requisicao.content

        # Conteúdo do site em formato de texto, ainda ilegível
        html_pagina = resposta_bytes.decode( decode )

        # Texto html corrido, separado por tags, perfeitamente legível
        return bs4.BeautifulSoup( html_pagina, 'html.parser' )

    # Caso falhe, apenas passe, não trave a execução do scrip geral
    except: pass

A seguir, um exemplo do que essa função faz.

Vamos entrar no site da câmara de vereadores e consultar uma página de uma proposição qualquer:

In [14]:
# Url para exemplo
exemplo = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=300&aaProposicao=2007'

# Visualizando página do link
IFrame( exemplo, width = '800', height = '400' )

Agora, **resumo** do conteúdo HTML da página:

In [15]:
# Usando a função criada
htmlSite( exemplo )

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="legis.ico" rel="shortcut icon" type="image/x-ico"/>
<title>Câmara Municipal de Indaial _ Indicação nº 300/2007 de 20/03/2007</title>
<meta content="Câmara Municipal de Indaial _ Indicação nº 300/2007 de 20/03/2007" name="description">
<link href="css/geral3.css" rel="stylesheet"/>
<link href="https://d11gitgevq44cw.cloudfront.net/libs/font-awesome/5x/css/all.min.css" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" rel="stylesheet"/>
<script crossorigin="anonymous" integrity="sha256-FgpCb/KJQlLNfOu91ta32o/NMZxltwRo8QtmkMRdAu8=" src="https://code.jquery.com/jquery-3.3.1.min.js"></script>
<script crossorigin="anonymous" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUi

### **Dados a serem extraídos**
<font size = '3'>
Tendo o HTML em mãos, podemos agora extrair seus dados.

No exemplo a seguir, veremos que nossos dados estarão armazenados nas tags 'dt' (colunas) e 'dd' (linha ou dados), as quais<br>
podemos encontrar usando o método '.find_all()'.
</font>


In [16]:
# Url para exemplo
exemplo = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=300&aaProposicao=2007'

# Conteúdo HTML do site
html = htmlSite( exemplo )

# Mostrando todas as tags 'dt' e 'dd' e seus respectivos conteúdos
print( html.find_all( 'dt' ) )
print( html.find_all( 'dd' ) )

[<dt class="col-sm-3">Reunião</dt>, <dt class="col-sm-3">Deliberação</dt>, <dt class="col-sm-3">Situação</dt>, <dt class="col-sm-3">Assunto</dt>, <dt class="col-sm-3">Autor</dt>]
[<dd class="col-sm-9">20/03/2007</dd>, <dd class="col-sm-9">20/03/2007</dd>, <dd class="col-sm-9">Proposição Aprovada</dd>, <dd class="col-sm-9">Abrigo de passageiros</dd>, <dd class="col-sm-9">Vereador <br/><b>Rubens Reinhold Ittner</b>.</dd>]


Para ficar mais visível os valores:

In [17]:
# Mostrando os respectivos conteúdos das tags 'dt' e 'dd'
print( [ i.get_text() for i in html.find_all( 'dt' ) ] )
print( [ i.get_text() for i in html.find_all( 'dd' ) ] )

['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor']
['20/03/2007', '20/03/2007', 'Proposição Aprovada', 'Abrigo de passageiros', 'Vereador Rubens Reinhold Ittner.']


Podemos verificar que é o mesmo conteúdo que conseguimos ver na página formatado como tabela.

### **Extração de dados do HTML**
<font size = '3'>
Essa atividade será executada pela função 'dadosProposicao', que extrairá dados de tags de <br>
'dt' e 'dd' e armazenará em um DataFrame pandas.<br>

A url das proposições segue um padrão, onde 1º temos o tipo de proposição, logo após o número dela e, por fim, <br>
o ano em que a respectiva foi realizada; dessa forma, com o uso de iterações podemos fazer requisições em larga <br>
escala e extrair os dados. <br>

Um ponto de obervação que devemos ter é a possibilidade de a página requisitada não retornar dados, por exemplo: <br>
no ano de 20xx tiveram 1.000 proposições, e na nossa requisição procuramos pela número 1.001... ela irá falhar. <br>
Dessa forma, identifiquei dois padrões: quando a proposição requisitada não existe ou não está disponível e armazenei <br>
na variável 'condicoes'. Após isso, com auxílio de um if, construí um bloco de verificação.<br>
</font>

In [18]:
# Função para, a partir do HTML do site da prefeitura, coletar dados das tags de interesse e armazenar em um DataFrame
def dadosProposicao( ano: int, num_proposicao: int, tipo_proposicao: int = 1 ) -> pd.DataFrame:

    """
    Argumentos:
        ano               (int): ano em que as proposições foram realizadas;
        num_proposicao    (int): Define a partir de qual proposição iniciará a extração;
        tipo_proposicao   (int): definir se a proposição é uma indicação (1), requerimento (2) ou moção (3).
    """

    # Url do site, para encontrar a preposição
    site = f'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao={ tipo_proposicao }&nrProposicao={ num_proposicao }&aaProposicao={ ano }'

    # HTML do site
    html = htmlSite( site )

    # VERIFICAÇÃO DE PROPOSIÇÃO INEXISTENTE ----------------------------------------------

    condicoes = ( ( html.find_all( 'dt' ) == []               ) or # Verifica se as tags onde guardam os nomes das colunas existem
                  ( 'Proposição não existente' in str( html ) ) )  # Verifica se a substring está contida no HTML

    # COLETANDO DADOS DO SITE ------------------------------------------------------------

    # Se a proposição não existir, retorne um erro no output; se existir, colete os dados
    if  condicoes: raise ValueError()

    else:
        # Criando estrutura do DataFrame esperado
        dfEstrutural = pd.DataFrame( columns = ['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'] )

        # Coletando apenas tags de interesse
        html_colunas = html.find_all( 'dt' ) # No HTML do site, 'dt' é a tag com o nome das colunas 
        html_linha   = html.find_all( 'dd' ) # No HTML do site, 'dd' é a tag em que o conteúdo da linha está contido

        # Gerando DataFrame com todos os textos contidos nas tags de interesse
        dados = pd.DataFrame( columns = [   i.get_text() for i in html_colunas  ],
                              data    = [ [ i.get_text() for i in html_linha  ] ] ) # É necessário envolver em lista dupla

        # Registrando informações que não estão nas tags consultadas
        dados['Proposição'] = num_proposicao
        dados['Ano'       ] = ano
        dados['Texto'     ] = html.p.get_text()

        # Concantenando para, caso a proposição não tiver dados de uma coluna, a estrutura da tabela não se alterar
        df = pd.concat( [ dfEstrutural, dados ] )

    return df.rename( columns = { 'Reunião'    : 'dt_reuniao',
                                  'Deliberação': 'dt_deliberacao',
                                  'Situação'   : 'situacao',
                                  'Assunto'    : 'assunto',
                                  'Autor'      : 'autor',
                                  'Proposição' : 'proposicao',
                                  'Ano'        : 'ano',
                                  'Texto'      : 'texto' } )

Para exemplificar o resultado da função criada, podemos observar uma tabela (DataFrame pandas), com os mesmos<br>
dados que vimos mais acima na página web, como também no conteúdo das tags HTML.

In [19]:
#Não impor limite de caracteres na esposição de tabelas do pandas
pd.options.display.max_colwidth = None

# Usando os mesmos dados dos exemplos anteriores
dadosProposicao( 2007, 300 )

,dt_reuniao,dt_deliberacao,situacao,assunto,autor,proposicao,ano,texto
0,20/03/2007,20/03/2007,Proposição Aprovada,Abrigo de passageiros,Vereador Rubens Reinhold Ittner.,300,2007,"Implantação de um abrigo de passageiros na esquina das ruas Ilse Pequena com Mal. Deodoro da Fonseca, bairro Warnow.*com cópia para Setor de Trânsito.\n"


### **Iteração: extração de dados em larga escala**
<font size = '3'>
Tendo acesso ao HTML e construído o código de extração de seus dados, o foco agora é direcionado <br>
para coleta em larga escala, tarefa que será executada pela função 'tabelaResultado'.<br>

Para isso, alguns pontos nortearam a construção do código construído:<br>
1. a mecânica de loop escolhida, no caso, a while será utilizada nesse projeto;<br>
2. a quantidade de vezes seguidas em que houver erro na requisição de proposições, para parar a execução e;<br>
3. a quantidade de tempo entre uma extração e outra, para não sobrecarregar o site. <br>

O resultado será um pd.DataFrame com os dados coletados.
</font>

In [20]:
# Função com loop para realizar consultas às páginas no site e gerar uma tabela com o resultado
def tabelaResultado( ano: int, inicia_em: int, qtd_requisicoes: int, tipo_proposicao: int = 1, erros_admissiveis: int = 3, seg_espera: float = 1):

    """
    Argumentos:
        ano                 (int): ano em que as proposições foram realizadas;
        tipo_proposicao     (int): definir se a proposição é uma indicação (1), requerimento (2) ou moção (3).
        inicia_em           (int): Define a partir de qual proposição iniciará a extração;
        qtd_requisicoes     (int): Define quantas requisições serão realizadas a partir da 'inicia_em';
        erros_admissiveis   (int): quantas vezes a requisição pode falhar até entender que não há mais dados.
        seg_espera        (float): quantidade de tempo de espera até realizar próxima execução.
    """

    # Variáveis do loop
    ult_consulta     = inicia_em + qtd_requisicoes - 1 # O número da última proposição a ser consultada
    erros_ocorridos  = 0
    lista_dataFrames = []
    qtd_consultas    = 1 # Quantidade de consultas realizadas

    # Iteração
    while inicia_em <= ult_consulta and erros_ocorridos < erros_admissiveis:

        try:
            # Se não falhar, retornará dicionário com dados da proposição
            dados = dadosProposicao(  ano, inicia_em, tipo_proposicao )

            # Adicionando dicionário na lista criada
            lista_dataFrames.append( dados )

        # Caso a função falhe, registrará o erro
        except: erros_ocorridos += 1
        pass

        # Registrando em tela a quantidade de consultas realizadas
        print( f'{ qtd_consultas } Consulta(s) efetuada(s). { seg_espera } segundos para próxima execução.' )

        # Variáveis incrementais: adicionando 1 a cada nova consulta
        inicia_em     += 1
        qtd_consultas += 1

        # Aguardar a quantidade de tempo definida para continuar o loop
        time.sleep( seg_espera )

    return pd.concat( lista_dataFrames )